In [3]:
import pandas as pd

df = pd.read_csv("../data/raw/telegram_messages.csv", encoding="utf-8")
df["Date"] = pd.to_datetime(df["Date"])
df.head()


,Channel Title,Channel Username,ID,Message,Date,Media Path,Views
0,Zemen Express®,@ZemenExpress,7007,💥💥...................................💥💥\n\n🎯 L...,2025-06-24 11:49:18,7007_media,2356
1,Zemen Express®,@ZemenExpress,7006,💥💥...................................💥💥\n\n🎯 L...,2025-06-24 11:49:01,7006_media,1903
2,Zemen Express®,@ZemenExpress,7005,💥💥...................................💥💥\n\n🎯 L...,2025-06-24 11:48:41,7005_media,1916
3,Zemen Express®,@ZemenExpress,7004,💥💥👀 ...........💥💥\n\n📌 Electric Charcoal Burne...,2025-06-23 14:55:46,7004_media,2712
4,Zemen Express®,@ZemenExpress,7000,💥💥👀 ...........💥💥\n\n📌 Electric Charcoal Burne...,2025-06-23 14:55:40,7000_media,2218


In [4]:
from datetime import timedelta

# Group by vendor/channel
vendor_groups = df.groupby("Channel Username")

vendor_stats = []

for vendor, group in vendor_groups:
    group = group.sort_values("Date")
    total_posts = len(group)
    total_days = (group["Date"].max() - group["Date"].min()).days
    weeks = total_days / 7 if total_days >= 7 else 1

    posts_per_week = total_posts / weeks
    avg_views = group["Views"].mean()

    top_post = group.loc[group["Views"].idxmax()]
    top_post_text = top_post["Message"]
    top_post_views = top_post["Views"]
    top_post_date = top_post["Date"]

    vendor_stats.append({
        "Vendor": vendor,
        "Channel Title": top_post["Channel Title"],
        "Posts/Week": round(posts_per_week, 2),
        "Avg Views/Post": round(avg_views, 2),
        "Top Post Text": top_post_text,
        "Top Post Views": top_post_views,
        "Top Post Date": top_post_date
    })

vendor_df = pd.DataFrame(vendor_stats)
vendor_df.head()


,Vendor,Channel Title,Posts/Week,Avg Views/Post,Top Post Text,Top Post Views,Top Post Date
0,@AwasMart,AwasMart-አዋስማርት🎁,36.65,5007.11,❇️Portable drying rack Clips cloth hanger\n\nል...,10591,2024-06-11 17:35:43
1,@Fashiontera,Fashion tera,5.43,20755.73,Jacket \nSize Available\nPrice 1400\n@Hiwe5266...,74719,2019-10-17 08:01:15
2,@Shewabrand,Shewa Brand,7.80,12263.16,NIKE lunar\nsize 40--45\nMEDE IN VIETNAM\nSHEW...,114819,2022-04-30 13:38:17
3,@ZemenExpress,Zemen Express®,24.70,9176.35,🎩😂👋☃️🌟🕯🥶👼\n\n🔤🔤🔤🔤🔤\n\n🔤🔤🔤🔤🔤🔤🔤🔤\n\n🎩 በአሉን ምክንያት...,19352,2023-12-28 12:12:06
4,@aradabrand2,ⒶⓇⒶⒹⒶ ⒷⓇⒶⓃⒹ,9.25,5068.59,"#አዲስ_የገባ\nAirforce Dior\nsize 40,41,42,43\n🚛 F...",26633,2021-03-11 10:58:34


In [6]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

model_path = "../models/xlmr-ner-amharic"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)
id2label = model.config.id2label


In [7]:
def extract_entities(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs).logits

    predictions = torch.argmax(outputs, dim=2)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    labels = [id2label[p.item()] for p in predictions[0]]

    entities = {"Product": [], "PRICE": []}
    current_entity = []
    current_type = None

    for token, label in zip(tokens, labels):
        if label.startswith("B-"):
            if current_entity:
                entities[current_type].append(" ".join(current_entity))
            current_entity = [token.replace("▁", "")]
            current_type = label[2:]
        elif label.startswith("I-") and current_type == label[2:]:
            current_entity.append(token.replace("▁", ""))
        else:
            if current_entity:
                entities[current_type].append(" ".join(current_entity))
                current_entity = []
                current_type = None

    if current_entity:
        entities[current_type].append(" ".join(current_entity))

    return entities


In [8]:
vendor_df["Extracted Products"] = vendor_df["Top Post Text"].apply(lambda x: extract_entities(x).get("Product", []))
vendor_df["Extracted Prices"] = vendor_df["Top Post Text"].apply(lambda x: extract_entities(x).get("PRICE", []))


In [9]:
import re

def extract_numeric_price(prices):
    if prices:
        for p in prices:
            match = re.search(r"\d+", p.replace(",", ""))
            if match:
                return int(match.group())
    return None

vendor_df["Top Post Price (ETB)"] = vendor_df["Extracted Prices"].apply(extract_numeric_price)


In [10]:
vendor_df["Lending Score"] = (
    vendor_df["Avg Views/Post"] * 0.5 +
    vendor_df["Posts/Week"] * 50
).round(2)


In [11]:
final_scorecard = vendor_df[[
    "Vendor", "Channel Title", "Avg Views/Post", "Posts/Week",
    "Top Post Price (ETB)", "Lending Score"
]].sort_values("Lending Score", ascending=False)

final_scorecard.reset_index(drop=True, inplace=True)
final_scorecard


,Vendor,Channel Title,Avg Views/Post,Posts/Week,Top Post Price (ETB),Lending Score
0,@ethio_brand_collection,EthioBrand®,43781.03,7.67,None,22274.02
1,@Fashiontera,Fashion tera,20755.73,5.43,None,10649.36
2,@marakibrand,ማራኪ ცЯﾑŋの™,16207.16,9.03,None,8555.08
3,@belaclassic,BELLA CLASSIC®,13472.19,6.61,None,7066.60
4,@Shewabrand,Shewa Brand,12263.16,7.80,None,6521.58
5,@ZemenExpress,Zemen Express®,9176.35,24.70,None,5823.18
6,@AwasMart,AwasMart-አዋስማርት🎁,5007.11,36.65,None,4336.06
7,@marakisat2,🇪🇹 📡 MARAKI BRANDS🌍🏆,7177.15,8.62,None,4019.58
8,@classybrands,CLASSY BRAND ®,5179.67,12.61,None,3220.34
9,@aradabrand2,ⒶⓇⒶⒹⒶ ⒷⓇⒶⓃⒹ,5068.59,9.25,None,2996.80


In [12]:
final_scorecard.to_csv("../data/raw/vendor_scorecard.csv", index=False)
